In [1]:

import random
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.feature_extraction.text import CountVectorizer
import torch
from torch.utils.data import Dataset, DataLoader,random_split
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

CFG = {
    'IMG_SIZE':128,
    'EPOCHS':5,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':64,
    'SEED':41,
    'TRAIN_RATE':0.5,
}
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

all_df = pd.read_csv('../train.csv')



In [2]:
all_df

,id,img_path,overview,cat1,cat2,cat3
0,TRAIN_00000,./image/train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,자연,자연관광지,항구/포구
1,TRAIN_00001,./image/train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,레포츠,육상 레포츠,골프
2,TRAIN_00002,./image/train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,음식,음식점,한식
3,TRAIN_00003,./image/train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,음식,음식점,한식
4,TRAIN_00004,./image/train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,음식,음식점,한식
...,...,...,...,...,...,...
16981,TRAIN_16981,./image/train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,음식,음식점,한식
16982,TRAIN_16982,./image/train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,숙박,숙박시설,모텔
16983,TRAIN_16983,./image/train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,숙박,숙박시설,모텔
16984,TRAIN_16984,./image/train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,레포츠,육상 레포츠,"야영장,오토캠핑장"


In [3]:
# df = all_df.copy()
# train_rate = 0.9
# min_num = 1
# label_column = "cat3"

def split_df(df,train_rate,min_num,label_column):

    label_list = list(set(list(df[label_column])))
    label_list.sort()


    ## 각 항목별 df의 index가져옴
    train_index_list = []
    val_index_list = []
    for label in label_list:
        index_list = list(df[df[label_column]==label].index)
        ## 필요하다면 여기서 인덱스 리스트를 셔플해도 됨
        
        index_len = len(list(df[df[label_column]==label].index))

        if index_len*train_rate > min_num:
            train_index_list = train_index_list + index_list[:int(index_len*train_rate)]
            val_index_list = val_index_list + index_list[int(index_len*train_rate):]

    train_df = df.iloc[train_index_list]

    val_df = df.iloc[val_index_list]

    return train_df,val_df
    

tr,val = split_df(all_df,0.9,1,"cat3")

In [4]:
tr

,id,img_path,overview,cat1,cat2,cat3
40,TRAIN_00040,./image/train/TRAIN_00040.jpg,"전라북도 익산시 금마면에서 매월 2, 7, 12, 17, 22, 27일에 개설되는 ...",쇼핑,쇼핑,5일장
280,TRAIN_00280,./image/train/TRAIN_00280.jpg,"복흥지역은 내장산 국립공원과 인접하여 가을이면 단풍을 즐길 수 있으며, 10월 중에...",쇼핑,쇼핑,5일장
430,TRAIN_00430,./image/train/TRAIN_00430.jpg,곡성기차마을 전통시장은 1956년 개설된 장옥형의 중형시장으로 매월 3·8일마다 ...,쇼핑,쇼핑,5일장
460,TRAIN_00460,./image/train/TRAIN_00460.jpg,옛날 시골장 정취는 없으나 아직도 시골 아낙네나 할머니들이 남새밭에서 재배한 무공해...,쇼핑,쇼핑,5일장
759,TRAIN_00759,./image/train/TRAIN_00759.jpg,1830년대 안동 지역에는 여러 개의 오일장이 있었다. 구체적으로는 부내장과 신당장...,쇼핑,쇼핑,5일장
...,...,...,...,...,...,...
14713,TRAIN_14713,./image/train/TRAIN_14713.jpg,두릅나무과에 딸린 낙엽활엽교목으로 우리나라를 비롯하여 중국 일본 등지에 널리 분포되...,자연,관광자원,희귀동.식물
14803,TRAIN_14803,./image/train/TRAIN_14803.jpg,미탄면에서 42번 국도를 타고 정선방면으로 2.8㎞ 이동한 후 백운삼거리에서 우회전...,자연,관광자원,희귀동.식물
15018,TRAIN_15018,./image/train/TRAIN_15018.jpg,"명진리 마을 앞 들 가운데 서 있는 이 나무는 높이 14m, 둘레 7.7m, 수관은...",자연,관광자원,희귀동.식물
15037,TRAIN_15037,./image/train/TRAIN_15037.jpg,"* 단종 유배생활의 벗, 영월의 관음송 *<br /><br />한강 상류지역인 영월...",자연,관광자원,희귀동.식물


In [5]:
val

,id,img_path,overview,cat1,cat2,cat3
14729,TRAIN_14729,./image/train/TRAIN_14729.jpg,양양은 지리적으로 설악산과 오대산이 연접한 곳으로 토질과 기후 특성으로 인하여 특산...,쇼핑,쇼핑,5일장
14742,TRAIN_14742,./image/train/TRAIN_14742.jpg,포항에서 영덕 방향으로 18km 정도 들어오면 청하소재지를 연결하는 청하 삼거리가 ...,쇼핑,쇼핑,5일장
14860,TRAIN_14860,./image/train/TRAIN_14860.jpg,여주장에 대하여 옛 문헌에 기록된 것은 없으나 고려시대에 들어와서 시장이 없는 곳에...,쇼핑,쇼핑,5일장
14871,TRAIN_14871,./image/train/TRAIN_14871.jpg,영동지역은 소백산맥과 노령산맥의 험준한 산세가 조화를 이루고 있으며 3도의 접경속에...,쇼핑,쇼핑,5일장
15061,TRAIN_15061,./image/train/TRAIN_15061.jpg,5일장으로 1일과 6일에 장이 선다. 다른 시장에 비해 큰 규모는 아니지만 인근 농...,쇼핑,쇼핑,5일장
...,...,...,...,...,...,...
16073,TRAIN_16073,./image/train/TRAIN_16073.jpg,영암방조제와 금호방조제가 완공되자 영암호와 금호호라는 담수호가 탄생했다. 산이반도는...,자연,관광자원,희귀동.식물
16542,TRAIN_16542,./image/train/TRAIN_16542.jpg,"강화군 화도면에 있는 장화리 마을은 조류생태계의 보고로도 유명한 갯벌이며, 낙조 명...",자연,관광자원,희귀동.식물
16555,TRAIN_16555,./image/train/TRAIN_16555.jpg,장불재는 무등산 정상으로 가는 길목에 위치하여 정상부의 주상절리들을 한눈에 바라볼 ...,자연,관광자원,희귀동.식물
16851,TRAIN_16851,./image/train/TRAIN_16851.jpg,"느티나무는 키가 26m, 가슴높이의 줄기둘레가 10m 이상까지 자라는 낙엽성 큰키나...",자연,관광자원,희귀동.식물


In [6]:
tr.groupby('cat3').size()

cat3
5일장             148
ATV               3
MTB               1
강                93
게스트하우스           74
               ... 
헬스투어             13
헹글라이딩/패러글라이딩      8
호수               27
홈스테이             51
희귀동.식물           79
Length: 128, dtype: int64

In [7]:
val.groupby('cat3').size()

cat3
5일장             17
ATV              1
MTB              1
강               11
게스트하우스           9
                ..
헬스투어             2
헹글라이딩/패러글라이딩     1
호수               3
홈스테이             6
희귀동.식물           9
Length: 128, dtype: int64

In [10]:
val.groupby('cat3').size()

cat3
5일장             17
ATV              1
MTB              1
강               11
게스트하우스           9
                ..
헬스투어             2
헹글라이딩/패러글라이딩     1
호수               3
홈스테이             6
희귀동.식물           9
Length: 128, dtype: int64